In [1]:
import pandas as pd
import time
from selenium.webdriver import Chrome
import re
import random
import time
from selenium.webdriver.chrome.options import Options
from textblob import TextBlob

timeDelay = random.randrange(6, 12)

In [2]:
# browser = Chrome()
# url = "https://www.instagram.com/"
# browser.get(url)

In [3]:
# #Retrieve every link from a page
# browser.find_elements_by_tag_name('a')
# # Retrieve a link from a page that has "likes" in its text
# browser.find_element_by_partial_link_text('likes')

In [4]:
def recent_posts(tag):
    """With the input of an account page, scrape the 25 most recent posts urls"""
    url = "https://www.instagram.com/explore/tags/" + tag +"/"
    browser.get(url)
    post = 'https://www.instagram.com/p/'
    post_links = []
    while len(post_links) < 1000:
        links = [a.get_attribute('href') for a in browser.find_elements_by_tag_name('a')]
        for link in links:
            if post in link and link not in post_links:
                post_links.append(link)
        scroll_down = "window.scrollTo(0, document.body.scrollHeight);"
        browser.execute_script(scroll_down)
        time.sleep(timeDelay)
    else:
        return post_links[:1000]

In [5]:
def users_recent_posts(username):
    """With the input of an account page, scrape the 25 most recent posts urls"""
    url = url = "https://www.instagram.com/" + username + "/"
    browser.get(url)
    post = 'https://www.instagram.com/p/'
    post_links = []
    while len(post_links) < 1000:
        links = [a.get_attribute('href') for a in browser.find_elements_by_tag_name('a')]
        for link in links:
            if post in link and link not in post_links:
                post_links.append(link)
        scroll_down = "window.scrollTo(0, document.body.scrollHeight);"
        browser.execute_script(scroll_down)
        time.sleep(timeDelay)
    else:
        return post_links[:1000]

In [6]:
def insta_url_to_img(url, filename="insta"):
    """
    Getting the actual photo file from an Instagram url
    Args:
    url: Instagram direct post url
    filename: file name for image at url
    Returns:
    image file, saved locally
    """
#     firefox_options = Options()
#     firefox_options.add_argument("--headless")
#     browser = Firefox(firefox_options=firefox_options)
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    browser = Chrome(chrome_options=chrome_options)
    browser.get(url)
    try: 
        image = browser.find_element_by_xpath(
            """//*[@id="react-root"]/section/main/div/div/article/div[1]/div/div/div[1]/img""").get_attribute('src').split(' ')[0]
        urllib.request.urlretrieve(image, filename + "-" + url + ".jpg")
    # If image is not a photo, print notice
    except:
        print("No image")

In [7]:
def find_hashtags(title):
    hashtags = re.findall('#[A-Za-z]+', title)
    return hashtags

In [8]:
def insta_details(urls):
    """Take a post url and return post details"""
    post_details = []
    for link in urls:
        print(link)
        browser.get(link)
        
        try:
            xpath_title = '//*[@id="react-root"]/section/main/div/div/article/div[2]/div[1]/ul/div/li/div/div/div[2]/span'
            title = browser.find_element_by_xpath(xpath_title).text
        except Exception as e:
            print(e)
            title=None
        text = TextBlob(title).detect_language()
        print(text)
        if(text == 'en'):
        
            try:
                # This captures the standard like/views count. 
                xpath_likes = '//*[@id="react-root"]/section/main/div/div/article/div[2]/section[2]/div/div/button/span'
                likes = browser.find_element_by_xpath(xpath_likes)
                if likes:
                    likes=likes.text
                else:
                    xpath_views = '//*[@id="react-root"]/section/main/div/div/article/div[2]/section[2]/div/span'
                    likes = browser.find_element_by_xpath(xpath_views)
                    if likes:
                        likes=likes.text
                    else:
                        likes=None

            except Exception as e:
                print(e)
                likes = None

            try:   
                age = browser.find_element_by_css_selector('a time').text
            except Exception as e:
                print(e)
                age=None

            try:
                xpath_comment = '//*[@id="react-root"]/section/main/div/div/article/div[2]/div[1]/ul/ul[1]/div/li/div/div/div[2]/span'
                comment = browser.find_element_by_xpath(xpath_comment).text
            except Exception as e:
                print(e)
                comment=None

            try:
                xpath_location = '//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[2]/div[2]/a'
                location = browser.find_element_by_xpath(xpath_location).text
            except Exception as e:
                print(e)
                location=None

            try:    
                insta_link = link.replace('https://www.instagram.com/p','')
            except Exception as e:
                print(e)
                insta_link=None

            try:
                xpath_username = '//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[1]/div[1]/h2/a'
                username = browser.find_element_by_xpath(xpath_username).text
            except Exception as e:
                print(e)
                username=None

            #image = insta_url_to_img(link)

            post_details.append({'link': insta_link, 'username': username, 'likes/views': likes,'age': age, 'title': title, 'comment': comment, 'location': location})
        else:
            pass
        time.sleep(timeDelay)
        
    return post_details

In [9]:
browser = Chrome()
insta_posts = recent_posts("petfood")
#insta_posts = users_recent_posts("petcare")

insta_post_details = insta_details(insta_posts)

data_frame = pd.DataFrame(insta_post_details)
data_frame['hashtags'] = data_frame['title'].apply(lambda x: find_hashtags(x))

data_frame.to_csv("instagram_posts_petfood.csv", index=False)
browser.quit()

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: chrome=79.0.3945.88)
